[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/unboxai/examples-gallery/blob/main/text-classification/sklearn/banking/demo-banking.ipynb)


# Banking chatbot using sklearn

This notebook illustrates how sklearn models can be upladed to the Unbox platform.

In [2]:
%%bash

if [ ! -e "requirements.txt" ]; then
    curl "https://raw.githubusercontent.com/unboxai/examples-gallery/main/text-classification/sklearn/banking/requirements.txt" --output "requirements.txt"
fi

In [2]:
!pip install -r requirements.txt

  Using cached scikit_learn-0.24.1-cp38-cp38-macosx_10_13_x86_64.whl (7.2 MB)
  Using cached scipy-1.9.0-cp38-cp38-macosx_12_0_universal2.macosx_10_9_x86_64.whl (58.1 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)


## Importing the modules and loading the dataset

In [3]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

We have stored the dataset on the following S3 bucket. If, for some reason, you get an error reading the csv directly from it, feel free to copy and paste the URL in your browser and download the csv file. Alternatively, you can also find the dataset on [HuggingFace](https://huggingface.co/datasets/banking77).

In [4]:
DATASET_URL = "https://unbox-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/text-classification/banking.csv"

In [5]:
data = pd.read_csv(DATASET_URL)
data.head()

,text,category
0,I am still waiting on my card?,card_arrival
1,What can I do if my card still hasn't arrived ...,card_arrival
2,I have been waiting over a week. Is the card s...,card_arrival
3,Can I track my card while it is in the process...,card_arrival
4,"How do I know if I will get my card, or if it ...",card_arrival


In [6]:
data['category'] = data['category'].astype('category')
data['label_code'] = data['category'].cat.codes

## Splitting the data into training and validation sets

In [7]:
# shuffling the data
data = data.sample(frac=1, random_state=42)  

training_set = data[:7000]
validation_set = data[7000:]

## Training and evaluating the model's performance

In [ ]:
sklearn_model = Pipeline([('count_vect', CountVectorizer(ngram_range=(1,2), stop_words='english')), 
                          ('lr', LogisticRegression(random_state=42))])
sklearn_model.fit(training_set['text'], training_set['label_code'])

In [ ]:
print(classification_report(validation_set['label_code'], sklearn_model.predict(validation_set['text'])))

## Unbox part!

### pip installing unboxapi

In [ ]:
!pip install unboxapi

### Instantiating the client

In [ ]:
import unboxapi

client = unboxapi.UnboxClient("YOUR_API_KEY_HERE")

### Creating a project on the platform

In [ ]:
from unboxapi.tasks import TaskType

project = client.create_or_load_project(name="Banking Project",
                                        task_type=TaskType.TextClassification,
                                        description="Evaluating ML approaches for a chatbot")

### Uploading the validation set

In [ ]:
# Getting the label list
label_dict = dict(zip(data.category.cat.codes, data.category))

label_list = [None] * len(label_dict)
for index, label in label_dict.items():
    label_list[index] = label

In [ ]:
from unboxapi.tasks import TaskType

dataset = project.add_dataframe(
    df=validation_set,
    class_names=label_list,
    label_column_name="label_code",
    text_column_name="text",
    commit_message="First commit!"
)

### Uploading the model

First, it is important to create a `predict_proba` function, which is how Unbox interacts with your model

In [ ]:
def predict_proba(model, text_list):
    return model.predict_proba(text_list)

Let's test the `predict_proba` function to make sure the input-output format is consistent with what Unbox expects:

In [ ]:
texts = ['some new text, sweet noodles', 'where is my card?', 'sad day']

predict_proba(sklearn_model, texts)

Now, we can upload the model:

In [ ]:
from unboxapi.models import ModelType

model = project.add_model(
    function=predict_proba, 
    model=sklearn_model,
    model_type=ModelType.sklearn,
    class_names=label_list,
    name='Banking Model',
    commit_message='First commit!',
    requirements_txt_file='requirements.txt'
)